<a href="https://www.kaggle.com/code/wolflxchuppy/spaceship-titanic?scriptVersionId=151629234" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


Objective: 
To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [3]:
data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
data.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [5]:
data.isnull().any()

PassengerId     False
HomePlanet       True
CryoSleep        True
Cabin            True
Destination      True
Age              True
VIP              True
RoomService      True
FoodCourt        True
ShoppingMall     True
Spa              True
VRDeck           True
Name             True
Transported     False
dtype: bool

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [7]:
data.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [8]:
data['Transported'] = data['Transported'].astype(int)

In [9]:
data['VIP'].fillna(0, inplace = True)
data['VIP'] = data['VIP'].astype(int)

In [10]:
data['CryoSleep'].fillna(0, inplace = True)
data['CryoSleep'] = data['CryoSleep'].astype(int)

In [11]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1


In [12]:
df = data[['PassengerId','CryoSleep','Age','VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Transported' ]]

In [13]:
df.isnull().any()

PassengerId     False
CryoSleep       False
Age              True
VIP             False
RoomService      True
FoodCourt        True
ShoppingMall     True
Spa              True
VRDeck           True
Transported     False
dtype: bool

In [14]:
df['Age'].fillna(0, inplace = True)
df['RoomService'].fillna(0, inplace = True)
df['FoodCourt'].fillna(0, inplace = True)
df['ShoppingMall'].fillna(0, inplace = True)
df['Spa'].fillna(0, inplace = True)
df['VRDeck'].fillna(0, inplace = True)

/tmp/ipykernel_19/1300137845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'].fillna(0, inplace = True)
/tmp/ipykernel_19/1300137845.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RoomService'].fillna(0, inplace = True)
/tmp/ipykernel_19/1300137845.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FoodCourt'].fillna(0, inplace = True)
/tmp/ipykernel_19/1300137845.py:4: SettingWithCopyWarning: 
A value is try

In [15]:
df.corr()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
PassengerId,1.000000,-0.006883,-0.005252,0.013608,0.000375,-0.009200,0.017795,-0.005198,0.015945,0.021491
CryoSleep,-0.006883,1.000000,-0.077098,-0.078281,-0.244089,-0.205928,-0.207798,-0.198307,-0.192721,0.460132
Age,-0.005252,-0.077098,1.000000,0.092662,0.069060,0.123475,0.036243,0.114818,0.096561,-0.071330
VIP,0.013608,-0.078281,0.092662,1.000000,0.056566,0.125499,0.018412,0.060991,0.123061,-0.037261
RoomService,0.000375,-0.244089,0.069060,0.056566,1.000000,-0.015126,0.052337,0.009244,-0.018624,-0.241124
FoodCourt,-0.009200,-0.205928,0.123475,0.125499,-0.015126,1.000000,-0.013717,0.221468,0.224572,0.045583
ShoppingMall,0.017795,-0.207798,0.036243,0.018412,0.052337,-0.013717,1.000000,0.014542,-0.007849,0.009391
Spa,-0.005198,-0.198307,0.114818,0.060991,0.009244,0.221468,0.014542,1.000000,0.147658,-0.218545
VRDeck,0.015945,-0.192721,0.096561,0.123061,-0.018624,0.224572,-0.007849,0.147658,1.000000,-0.204874
Transported,0.021491,0.460132,-0.071330,-0.037261,-0.241124,0.045583,0.009391,-0.218545,-0.204874,1.000000


In [16]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [17]:
X= df.iloc[:,0:8]   #all features
Y= df.iloc[:,-1]   #target output (Transported)

In [18]:
best_features= SelectKBest(score_func=chi2, k=3)
fit= best_features.fit(X,Y)

In [19]:
df_scores= pd.DataFrame(fit.scores_)
df_columns= pd.DataFrame(X.columns)

In [20]:
features_scores= pd.concat([df_columns, df_scores], axis=1)
features_scores.columns= ['Features', 'Score']
features_scores.sort_values(by = 'Score')

,Features,Score
3,VIP,1.179281e+01
2,Age,3.483062e+02
1,CryoSleep,1.197498e+03
6,ShoppingMall,1.616605e+03
5,FoodCourt,1.025621e+05
0,PassengerId,6.181576e+05
4,RoomService,1.002141e+06
7,Spa,1.726733e+06


In [21]:
X_Train= df[['PassengerId', 'Spa', 'RoomService','FoodCourt']] # the top 3 features
Y_train= df[['Transported']]  #the target output

In [22]:
logreg= LogisticRegression()
logreg.fit(X_Train,Y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [23]:
data2 = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
data2.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [24]:
X_test= data2[['PassengerId','Spa', 'RoomService','FoodCourt']]

In [25]:
X_test['RoomService'].fillna(0, inplace = True)
X_test['Spa'].fillna(0, inplace = True)
X_test['FoodCourt'].fillna(0, inplace = True)

/tmp/ipykernel_19/1091336242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['RoomService'].fillna(0, inplace = True)
/tmp/ipykernel_19/1091336242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Spa'].fillna(0, inplace = True)
/tmp/ipykernel_19/1091336242.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['FoodCourt'].fillna(0, inplace = True)


In [26]:
y_pred=logreg.predict(X_test)

In [27]:
print (X_test) #test dataset
print (y_pred) #predicted values

     PassengerId     Spa  RoomService  FoodCourt
0        0013_01     0.0          0.0        0.0
1        0018_01  2823.0          0.0        9.0
2        0019_01     0.0          0.0        0.0
3        0021_01   181.0          0.0     6652.0
4        0023_01     0.0         10.0        0.0
...          ...     ...          ...        ...
4272     9266_02     0.0          0.0        0.0
4273     9269_01    10.0          0.0      847.0
4274     9271_01     0.0          0.0        0.0
4275     9273_01     0.0          0.0     2680.0
4276     9277_01     0.0          0.0        0.0

[4277 rows x 4 columns]
[1 0 1 ... 1 1 1]


In [28]:
y_pred=logreg.predict(X_test)

In [29]:
print (X_test) #test dataset
print (y_pred) #predicted values

     PassengerId     Spa  RoomService  FoodCourt
0        0013_01     0.0          0.0        0.0
1        0018_01  2823.0          0.0        9.0
2        0019_01     0.0          0.0        0.0
3        0021_01   181.0          0.0     6652.0
4        0023_01     0.0         10.0        0.0
...          ...     ...          ...        ...
4272     9266_02     0.0          0.0        0.0
4273     9269_01    10.0          0.0      847.0
4274     9271_01     0.0          0.0        0.0
4275     9273_01     0.0          0.0     2680.0
4276     9277_01     0.0          0.0        0.0

[4277 rows x 4 columns]
[1 0 1 ... 1 1 1]


In [30]:
col_titles = ['prediction']
y_pred = pd.DataFrame(y_pred, columns=col_titles)
y_pred.replace({1: True, 0: False}, inplace=True)
y_pred

,prediction
0,True
1,False
2,True
3,True
4,False
...,...
4272,True
4273,True
4274,True
4275,True


In [31]:
x_test = X_test[['PassengerId']]

In [32]:
df_combined = pd.concat([x_test, y_pred], axis=1, ignore_index=True)
# Add column headers to the combined DataFrame
column_headers = ['PassengerId', 'Transported']  # Adjust these to your actual column names

# Option 1: Set column headers when creating the DataFrame
df_combined.columns = column_headers

In [33]:
df_combined

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [34]:
df_combined.to_csv('/kaggle/working/submission.csv', index=False)